# Tensorflow 101: Introduction to low-level API

*Prequisites*:
1. Sudah menginstal tensorflow
2. Minat dan semangat
3. Sudah dengerin kami presentasi tentang pengantar dari tensorflow bagian pertama


### Cek *dependencies*

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
tf.__version__

'1.11.0'

*seems okay*

# Simple Computational Graph


Membuat grafik penjumlahan. **ingat: build then running**. Pertama yang perlu dilakukan adalah membuat dahulu *computational graph*nya.

Namun untuk kesempatan pertama ini, kita mencoba dengan menggunakan fungsi ``tf.constant`` untuk membuat konstanta.


### tf.constant
```
tf.constant(
    value,
    dtype=None,
    shape=None,
    name='Const',
    verify_shape=False
)
```


Untuk membuat konstanta. Hasilnya adalah tensor dengan nilai yang sesuai dengan ``dtype`` dan bernilai sama dengan ``value`` dengan ukuran (opsional) dengan ``shape``

[Dokumentasi dalam bentuk kode](https://github.com/tensorflow/tensorflow/blob/a6d8ffae097d0132989ae4688d224121ec6d8f35/tensorflow/python/framework/constant_op.py#L118)

[Dokumentasi dari tf](https://www.tensorflow.org/api_docs/python/tf/constant)

In [3]:
tensor = tf.constant([1, 2, 3, 4, 5, 6, 7])
tensor.__dict__

{'_op': <tf.Operation 'Const' type=Const>,
 '_value_index': 0,
 '_dtype': tf.int32,
 '_shape_val': None,
 '_consumers': [],
 '_id': 1}

In [4]:
print(tensor) # Tensor dengan konstanta nilai [1 2 3 4 5 6 7]
print(type(tensor))

Tensor("Const:0", shape=(7,), dtype=int32)
<class 'tensorflow.python.framework.ops.Tensor'>


Dari hasil cetak di atas tidak mengnampilkan nilai karena sebenarnya pada tf yang dibangung ada *computational graph*nya. Objek ``Tensor`` tersebut hanya merepresentasikan model yang sedang dirancang

In [5]:
var_a = tf.constant([1.0,2.0], dtype=tf.float32, name='var_a')
print(var_a)

Tensor("var_a:0", shape=(2,), dtype=float32)


Oke, setelah paham dengan ``tf.constant`` maka selanjutnya kita bisa membuat *graph*nya

## Build the graph

In [6]:
a = tf.constant(5.0, dtype=tf.float32)
b = tf.constant(9.0) # Secara tidak langsung punya tipe data yang sama
total = a + b # tf bisa langsung dijumlah
print(a)
print(b)
print(total)

Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("Const_2:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


### Tensorboard

Sebuah fungsi yang mempermudah pengguna untuk melihat graph yang telah dibuat.

Untuk menjalankannya, ketik

``` tensorboard --logdir .```

pada terminal/cmd **setelah menjalankan potongan program dibawah**

In [7]:
writer = tf.summary.FileWriter('.') # Nama berkas dapat diganti, 
# begitu juga dengan shell command-nya
writer.add_graph(tf.get_default_graph())

Buka ```localhost:6006``` (secara *default*) maka akan didapati gambar kurang lebih seperti ini:

![sim_graph](sim_com_gr_tf.PNG "sim graph")


## *Compile! (Run the graph)*

In [8]:
sess = tf.Session()
print(sess.run(total))

14.0


Pada saat dijalankan, semua ``tf.Tensor`` hanya memilki nilai tunggal (satu jenis nilai) meski telah menggunakan fungsi random. Sebagai contoh kita memanggil fungsi ``tf.random_uniform`` yang akan menghasil vektor dengan 5 elemen kemudian menjalakan *session*nya.

In [9]:
vec = tf.random_uniform(shape=(5,))
out1 = vec + 1
out2 = vec + 2
out3 = vec + 3
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2, out3))) # single run

[0.17893672 0.9285165  0.6016029  0.4783088  0.03498745]
[0.15033066 0.12529778 0.33547032 0.58683026 0.12419474]
(array([1.1877897, 1.9836812, 1.142723 , 1.3845692, 1.7159133],
      dtype=float32), array([2.1877897, 2.9836812, 2.142723 , 2.3845692, 2.7159133],
      dtype=float32), array([3.1877897, 3.9836812, 3.142723 , 3.3845692, 3.7159133],
      dtype=float32))


Dari hasil diatas menunjukkan bahwa dalam *single run* nilai yang dihasilkan dari ``tf.random_uniform`` hanya memiliki satu jenis nilai.


**Jadi ini aja?** *ngga*

#### Latihan 1
Kalau sudah paham dengan penggunaan konstanta tadi, coba kerjakan latihan berikut:
1. Buat graph perkalian array/list
2. Apa yang dilakukan oleh potongan script berikut? **Jangan dahulu dijalankan**

```
x = tf.constant([1, 2, 3])
x_sum = tf.reduce_sum(x)
result = x_sum/tf.size(x)
```

3. Jalankan program diatas untuk melihat hasilnya! *feel free*, buat ganti nilainya

In [10]:
# Nomor 3 disini

### *Feeding with tf.placeholder* (Ngasih makan; kurang lebih) 

Ngga cuma dengan konstanta aja, graph yang dirancang juga bisa menerima input menggunakan fungsi ``tf.placeholder``. Fungsi ini mirip dengan membuat fungsi pada python. Keterangannya sebagai berikut:

Sebagai contoh:

In [11]:
# Membut graph perkalian
var_a = tf.placeholder(name='A', dtype=tf.float32)
var_b = tf.placeholder(name='B', dtype=tf.float32)
result = var_a*var_b

Sekarang coba isi dengan nilai. Yang perlu diperhatikan, saat mengisi ``feed_dict``, nilai ``key`` yang ditulis adalah nama dari *placeholder*nya.

In [12]:
print(sess.run(result, feed_dict={var_a: 4, var_b: 5}))

20.0


In [13]:
# Cara lain. Cari tahu dahulu nama dari masing-masing placeholder
print(var_a.name)
print(var_b.name)

A:0
B:0


In [15]:
print(sess.run(result, feed_dict={sess.graph.get_tensor_by_name('A:0'): 4, 
                                  sess.graph.get_tensor_by_name('B:0'): 5}))

20.0


In [16]:
# Input array
print(sess.run(result, feed_dict={var_a: [3, 4, 5], var_b: [1, 3, 89]}))

[  3.  12. 445.]


**Paham?** *sekarang soal lagi*

#### Latihan 2
Fungsi nomor 1 & 2 di latihan sebelumnya dibuat agar dapat menerima input dari *user* menggunakan fungsi ``input()`` bawaan dari python!

In [17]:
# Kerjakan disini!1!11!!!

### Datasets

Kalau tadi sudah paham dengan fungsi dari ``tf.placeholder`` untuk melakukan eksperimen, maka ada cara lain yang lebih 'dibenarkan' oleh *tensorflow* untuk melakukan *streaming data* ke graph.

Langkah kerja dari *streaming data* menggunakan `tf.data`:
1. Buat variabel *slices* atau iterator untuk memotong data
2. Panggil iterator ke session
3. Ketika sudah mencapai akhir dari baris, maka akan menghasilkan error. Oleh sebab itu harus menggunakan metode *try-catch-exception*
4. Langkah 2-3 dilakukan dalam perulangan *while* (umumnya)

In [18]:
data_saya = [
    [2, 49,],
    [4, 78,],
    [5, 6,],
    [9, 12,],
]
slices = tf.data.Dataset.from_tensor_slices(data_saya)
next_item = slices.make_one_shot_iterator().get_next()
count =0
while True:
  try:
    print('Iterasi ke-', count)
    print(sess.run(next_item))
    count+=1
  except tf.errors.OutOfRangeError:
    break

Iterasi ke- 0
[ 2 49]
Iterasi ke- 1
[ 4 78]
Iterasi ke- 2
[5 6]
Iterasi ke- 3
[ 9 12]
Iterasi ke- 4


Contoh berikut ini jika dataset bergantung pada suatu *state* atau keadaan. Misal harus menghasilkan nilai random

In [19]:
r = tf.random_normal([10,3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()
count=0
sess.run(iterator.initializer)
while True:
  try:
    count+=1
    print('Iterasi ke-', count)
    print(sess.run(next_row))
  except tf.errors.OutOfRangeError:
    break

Iterasi ke- 1
[-0.54027236  0.65042603  0.6567037 ]
Iterasi ke- 2
[-1.2551336 -1.4917736  1.1707697]
Iterasi ke- 3
[0.8733775  0.34707755 0.07027091]
Iterasi ke- 4
[ 1.1168051  -0.59571296 -1.1088066 ]
Iterasi ke- 5
[-0.8761318  0.503862  -1.3557749]
Iterasi ke- 6
[-0.5658852 -0.4508208 -0.4148615]
Iterasi ke- 7
[ 1.1668557   1.44209    -0.06521785]
Iterasi ke- 8
[-0.0387431  -0.88227946  0.21499534]
Iterasi ke- 9
[ 0.06847714 -0.40781102 -1.2648095 ]
Iterasi ke- 10
[-0.01374114  0.3401839  -0.1934982 ]
Iterasi ke- 11


**Paham?** *ngga, ayo soal lagi*

#### Latihan 3
Fungsi nomor 2 di latihan 3 sebelumnya dibuat agar menghasilkan nilai dari array ``data_saya``

In [20]:
# Hint: Hanya operasi resultnya saja yang digunakan
data_saya = [
    [2, 49,],
    [4, 78,],
    [5, 6,],
    [9, 12,],
]
slices = tf.data.Dataset.from_tensor_slices(data_saya)
iterator = dataset.make_initializable_iterator()
next_item = iterator.get_next()
next_item_int = tf.cast(next_item, tf.int32)
result = next_item_int/tf.size(next_item)
count =0
sess.run(iterator.initializer)
while True:
  try:
    print('Iterasi ke-', count)
    print(sess.run(next_item))
    print(sess.run(result))
    count+=1
  except tf.errors.OutOfRangeError:
    break

Iterasi ke- 0
[-0.15253161  0.6167842  -1.9662676 ]
[0. 0. 0.]
Iterasi ke- 1
[-0.42447257 -0.12695111 -1.8894917 ]
[0.         0.         0.33333333]
Iterasi ke- 2
[-0.19169414 -1.7749436   1.4285809 ]
[0. 0. 0.]
Iterasi ke- 3
[ 0.47504345 -1.1214664  -0.01972046]
[0. 0. 0.]
Iterasi ke- 4
[-0.9895589  1.5724908 -0.9600632]
[0.66666667 0.         0.        ]
Iterasi ke- 5


### Layers
*Finally, layers! but not in this discussion* 

**sorry!**